# Get stock price

In [1]:
import pandas as pd
import requests

from tqdm import tqdm
from datetime import datetime, timedelta


In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

def get_friday_dates(start_date, end_date):
    start_date = datetime.strptime(start_date, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y%m%d")
    friday_dates = []

    while start_date <= end_date:
        if start_date.weekday() == 4:
            friday_dates.append(start_date.strftime("%Y%m%d"))
        start_date += timedelta(days=1)
    
    return friday_dates

fridays = get_friday_dates("20180101", "20240630")

In [3]:
x = pd.DataFrame()
for date in tqdm(fridays):
    url = f'https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date={date}&type=ALL&response=csv'
    res = requests.get(url, headers=headers)
    data = res.text
    s_data = data.split('\n')
    output = []
    
    for d in s_data:
        _d = d.split('","')
        length = len(_d)
        symbol = _d[0]
        if length == 16 and symbol == '\"2379':
            df = pd.DataFrame([_d])
            df['date'] = date
            x = pd.concat([x, df], ignore_index=True)



  0%|          | 0/339 [00:00<?, ?it/s]

100%|██████████| 339/339 [11:36<00:00,  2.06s/it]


In [4]:
df = x

In [5]:
new_columns = ["證券代號", "證券名稱", "成交股數", "成交筆數", "成交金額", "開盤價", "最高價", "最低價", "收盤價", "漲跌(+/-)", "漲跌價差", "最後揭示買價", "最後揭示買量", "最後揭示賣價", "最後揭示賣量", "本益比", "date"]

In [6]:
df.columns = new_columns
df = df.iloc[:, [0, 1, 8, 16]]

In [ ]:
x = df
x

In [8]:
x.to_csv("x_stock_price.csv", index=False)